In [1]:
import json
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
data = []
with open('../logs/20240327_1620.log', 'r') as fp:
    raw = fp.read()
    for ln in raw.split('\n'):
        if not ln:
            continue
        data.append(json.loads(ln))

In [3]:
def extract_fields(message_dict):
    fields = {}
    
    # Extract the required fields
    fields['level'] = message_dict['record']['level']['name']
    fields['timestamp'] = message_dict['record']['time']['repr']
    fields['message'] = message_dict['record']['message']
    fields['thread_id'] = message_dict['record']['thread']['id']
    fields['process'] = message_dict['record']['process']['name']
    fields['conversation_id'] = message_dict['record']['extra'].get('conversation_id', None)
    fields['turn'] = message_dict['record']['extra'].get('turn', None)
    
    return fields

In [4]:
df = [extract_fields(d) for d in data]
df = pd.DataFrame(df)
df['timestamp'] = pd.to_datetime(df.timestamp)
df = df.sort_values('timestamp')
df['timediff'] = df.timestamp - df.timestamp.shift(1)
df['timediff'] = df.timediff.apply(lambda x: x.seconds)

In [5]:
def get_input_toks(x):
    if 'Tokens:' in x:
        return int(x.split(',')[-2].strip())
    return int(-1)

def get_output_toks(x):
    if 'Tokens:' in x:
        return int(x.split(',')[-1].strip().strip(')'))
    return int(-1)

In [6]:
df['input_toks'] = df.message.apply(get_input_toks)
df['output_toks'] = df.message.apply(get_output_toks)
df['total_toks'] = df.input_toks + df.output_toks

In [50]:
df.query('turn == 1 & input_toks == -1')

,level,timestamp,message,thread_id,process,conversation_id,turn,timediff,input_toks,output_toks,total_toks
78,INFO,2024-03-27 16:20:18.616031+05:30,Conversation turn for conversation id 2\t1,134527992747072,ForkProcess-3,2.0,1.0,0.0,-1,-1,-2
80,INFO,2024-03-27 16:20:19.343167+05:30,Conversation turn for conversation id 9\t1,134527992747072,ForkProcess-10,9.0,1.0,0.0,-1,-1,-2
82,INFO,2024-03-27 16:20:19.489736+05:30,Conversation turn for conversation id 6\t1,134527992747072,ForkProcess-7,6.0,1.0,0.0,-1,-1,-2
84,INFO,2024-03-27 16:20:19.508438+05:30,Conversation turn for conversation id 0\t1,134527992747072,ForkProcess-1,0.0,1.0,0.0,-1,-1,-2
86,INFO,2024-03-27 16:20:19.874362+05:30,Conversation turn for conversation id 12\t1,134527992747072,ForkProcess-13,12.0,1.0,0.0,-1,-1,-2
88,INFO,2024-03-27 16:20:20.319913+05:30,Conversation turn for conversation id 5\t1,134527992747072,ForkProcess-6,5.0,1.0,0.0,-1,-1,-2
90,INFO,2024-03-27 16:20:20.423238+05:30,Conversation turn for conversation id 13\t1,134527992747072,ForkProcess-14,13.0,1.0,0.0,-1,-1,-2
92,INFO,2024-03-27 16:20:20.656238+05:30,Conversation turn for conversation id 1\t1,134527992747072,ForkProcess-2,1.0,1.0,0.0,-1,-1,-2
94,INFO,2024-03-27 16:20:20.675248+05:30,Conversation turn for conversation id 4\t1,134527992747072,ForkProcess-5,4.0,1.0,0.0,-1,-1,-2
96,INFO,2024-03-27 16:20:20.975190+05:30,Conversation turn for conversation id 3\t1,134527992747072,ForkProcess-4,3.0,1.0,0.0,-1,-1,-2


In [46]:
metrics = {
    'num_turns': [],
    'e2e_time': [],
    'prompt_tokens': [],
    'output_tokens': [],
    'mean_ip_tokens': [],
    'mean_op_tokens': []
}

for _, gp in df.groupby('conversation_id'):
    temp = gp.sort_values('timestamp')
    num_turns = temp.turn.max() + 1
    if num_turns == 1:
        break
    e2e_time = temp.query('turn.isna()').timestamp.iloc[-1] - temp.query('turn.notna()').timestamp.iloc[0]
    e2e_time = e2e_time.seconds
    prompt_tokens = temp.query('input_toks != -1').input_toks.iloc[-1]
    output_tokens = temp.query('input_toks != -1').output_toks.iloc[-1]
    ip_tokens = temp.query('input_toks != -1 & turn.notna()').input_toks - temp.query('input_toks != -1 & turn.notna()').total_toks.shift(1)
    ip_tokens = ip_tokens.mean()
    op_tokens = temp.query('input_toks != -1 & turn.notna()').output_toks.mean()

    metrics['num_turns'].append(num_turns)
    metrics['e2e_time'].append(e2e_time)
    metrics['prompt_tokens'].append(prompt_tokens)
    metrics['output_tokens'].append(output_tokens)
    metrics['mean_ip_tokens'].append(ip_tokens)
    metrics['mean_op_tokens'].append(op_tokens)

In [47]:
metrics

{'num_turns': [14.0,
  14.0,
  14.0,
  14.0,
  14.0,
  14.0,
  14.0,
  14.0,
  14.0,
  14.0,
  14.0,
  14.0,
  14.0,
  14.0,
  14.0],
 'e2e_time': [264,
  240,
  251,
  253,
  243,
  240,
  249,
  267,
  263,
  258,
  244,
  267,
  261,
  265,
  265],
 'prompt_tokens': [7295,
  7098,
  7114,
  7114,
  7200,
  7159,
  7106,
  7450,
  7265,
  7155,
  7146,
  7352,
  7232,
  7151,
  7216],
 'output_tokens': [512,
  512,
  512,
  512,
  512,
  503,
  512,
  359,
  512,
  456,
  512,
  512,
  512,
  512,
  498],
 'mean_ip_tokens': [148.08333333333334,
  148.0,
  148.16666666666666,
  148.0,
  148.08333333333334,
  148.08333333333334,
  148.16666666666666,
  148.25,
  148.16666666666666,
  148.0,
  148.08333333333334,
  148.16666666666666,
  148.08333333333334,
  148.08333333333334,
  148.08333333333334],
 'mean_op_tokens': [461.38461538461536,
  446.3076923076923,
  447.38461538461536,
  447.53846153846155,
  454.0769230769231,
  450.2307692307692,
  446.7692307692308,
  461.38461538461536,

In [43]:
temp.query('input_toks != -1')

,level,timestamp,message,thread_id,process,conversation_id,turn,timediff,input_toks,output_toks,total_toks
105,INFO,2024-03-27 16:20:22.823180+05:30,"Tokens: (544, 32, 512)",134527992747072,ForkProcess-15,14.0,0.0,0.0,32,512,544
135,INFO,2024-03-27 16:20:34.528121+05:30,"Tokens: (1124, 612, 512)",134527992747072,ForkProcess-15,14.0,1.0,0.0,612,512,1124
165,INFO,2024-03-27 16:20:47.758362+05:30,"Tokens: (1661, 1149, 512)",134527992747072,ForkProcess-15,14.0,2.0,0.0,1149,512,1661
193,INFO,2024-03-27 16:21:01.891014+05:30,"Tokens: (2215, 1728, 487)",134527992747072,ForkProcess-15,14.0,3.0,0.0,1728,487,2215
221,INFO,2024-03-27 16:21:16.762416+05:30,"Tokens: (2719, 2252, 467)",134527992747072,ForkProcess-15,14.0,4.0,1.0,2252,467,2719
253,INFO,2024-03-27 16:21:33.189725+05:30,"Tokens: (3201, 2743, 458)",134527992747072,ForkProcess-15,14.0,5.0,1.0,2743,458,3201
283,INFO,2024-03-27 16:21:51.801398+05:30,"Tokens: (3830, 3318, 512)",134527992747072,ForkProcess-15,14.0,6.0,0.0,3318,512,3830
313,INFO,2024-03-27 16:22:13.960062+05:30,"Tokens: (4545, 4167, 378)",134527992747072,ForkProcess-15,14.0,7.0,1.0,4167,378,4545
341,INFO,2024-03-27 16:22:38.494312+05:30,"Tokens: (5573, 5137, 436)",134527992747072,ForkProcess-15,14.0,8.0,1.0,5137,436,5573
370,INFO,2024-03-27 16:23:08.955979+05:30,"Tokens: (6207, 5695, 512)",134527992747072,ForkProcess-15,14.0,9.0,0.0,5695,512,6207


In [36]:
temp.query('turn.notna() & input_toks != -1').input_toks - temp.query('turn.notna() & input_toks != -1').total_toks.shift(1)

105      NaN
135     68.0
165     25.0
193     67.0
221     37.0
253     24.0
283    117.0
313    337.0
341    592.0
370    122.0
409    121.0
431    242.0
509     25.0
dtype: float64

In [ ]:
temp.query('turn.notna()').timestamp.iloc[0]

In [ ]:
temp.query('turn.isna()').timestamp.iloc[-1]

In [ ]:
np.mean(metrics['e2e_time'])

In [ ]:
np.mean(metrics['prompt_tokens'])

In [ ]:
np.mean(metrics['output_tokens'])

- Given a 7B model, 4090 GPU
- 0.2 requests per second for 1000 requests
- TTOP, TTFT